In [ ]:
!pip install ultralytics

import cv2
import os
from ultralytics import YOLO
from collections import Counter

# ----------------------------
# Paths
# ----------------------------
VIDEO_PATH = "/content/drive/MyDrive/IOT/License Plate detection model/side/1-5.mp4"
HELMET_RIDER_MODEL_PATH = "/content/drive/MyDrive/IOT/Helmet Models/best.pt"  # multi-class: rider+helmet
OUTPUT_DIR = "/content/drive/MyDrive/IOT/bike_frames"
OUTPUT_TXT = os.path.join(OUTPUT_DIR, "rider_helmet_counts.txt")
FINAL_OUTPUT_TXT = os.path.join(OUTPUT_DIR, "final_counts.txt")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ----------------------------
# Load model
# ----------------------------
model = YOLO(HELMET_RIDER_MODEL_PATH)

# ----------------------------
# Open video
# ----------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
frame_id = 0
saved_frames = 0
frame_info = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection
    results = model(frame, conf=0.5)

    for r in results:
        counts = r.boxes.cls.cpu().numpy()  # class IDs
        names = r.names  # class names

        # Count riders and helmets
        rider_count = sum(1 for c in counts if names[int(c)] == "Bike_Rider")
        helmet_count = sum(1 for c in counts if names[int(c)] == "Helmet")

        # Only save frames with at least one rider
        if rider_count > 0:
            save_path = os.path.join(OUTPUT_DIR, f"frame{frame_id}.jpg")
            cv2.imwrite(save_path, frame)
            saved_frames += 1

            # Record counts only for saved frames
            frame_info.append(f"frame{frame_id}.jpg\t{rider_count}\t{helmet_count}")

    frame_id += 1

cap.release()

# ----------------------------
# Save TXT with counts for saved frames only
# ----------------------------
with open(OUTPUT_TXT, "w") as f:
    f.write("frame\triders\thelmets\n")
    f.write("\n".join(frame_info))

print(f"✅ Done! Saved {saved_frames} frames with riders in: {OUTPUT_DIR}")
print(f"✅ Rider & helmet counts saved in: {OUTPUT_TXT}")

# ----------------------------
# Apply majority voting
# ----------------------------
rider_counts = []
helmet_counts = []

with open(OUTPUT_TXT, "r") as f:
    next(f)  # skip header
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) != 3:
            continue
        _, riders, helmets = parts
        rider_counts.append(int(riders))
        helmet_counts.append(int(helmets))

# Get most common counts
final_riders = Counter(rider_counts).most_common(1)[0][0]
final_helmets = Counter(helmet_counts).most_common(1)[0][0]

# Save final counts
with open(FINAL_OUTPUT_TXT, "w") as f:
    f.write("final_riders\tfinal_helmets\n")
    f.write(f"{final_riders}\t{final_helmets}\n")

print(f"✅ Majority voting done! Final counts saved in: {FINAL_OUTPUT_TXT}")
print(f"Riders: {final_riders}, Helmets: {final_helmets}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

0: 384x640 (no detections), 1481.1ms
Speed: 19.6ms preprocess, 1481.1ms inference, 17.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1280.0ms
Speed: 2.8ms preprocess, 1280.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1275.3ms
Speed: 2.3ms preprocess, 1275.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1562.5ms
Speed: 2.4ms preprocess, 1562.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 1437.1ms
Speed: 4.3ms preprocess,